In [ ]:
""" """

import enum
import strenum
import pathlib

import numpy as np
import pandas as pd

In [ ]:
@enum.unique
class C(strenum.StrEnum):
    STUDENT_NUMBER = "Student #"
    STUDENT_NAME = "Student Name"
    GRADE = "Grade"
    HOMEROOM = "Homeroom"
    PRICE = "Price"
    PAID = "Paid"
    ITEM_NAME = "Item Name"

    # Added by me
    ENVELOPE_NUMBER = "Envelope Number"
    CARD_NUMBER = "Card Number"

In [ ]:
base_path = pathlib.Path("thad/uhill_pac/grocery_card_artifacts/orders_2025_Q4")
files = list(base_path.rglob("*.xlsx"))
dfs = [pd.read_excel(f, header=13, dtype="str") for f in files]
for df in dfs:
    assert C.STUDENT_NUMBER in df.columns
df = pd.concat(dfs, axis="index")

In [ ]:
df = df[df[C.STUDENT_NUMBER].notna()]
df = df.dropna(axis="columns", how="all")
df

In [ ]:
df[C.ITEM_NAME].value_counts().sort_index()

In [ ]:
totals = (
    df.groupby([C.ITEM_NAME, C.PRICE])
    .size()
    .rename("Count")
    .reset_index()
    .set_index(C.ITEM_NAME)
    .astype(int)
)
totals["Total Price"] = totals[C.PRICE] * totals["Count"]
totals

In [ ]:
totals.query("`Item Name`.str.contains('Choices')")["Total Price"].sum()

In [ ]:
totals.query("`Item Name`.str.contains('Save-On-Foods')")["Total Price"].sum()

In [ ]:
# Make sure the price is right!
for _, row in df.iterrows():
    assert row[C.ITEM_NAME].endswith(row[C.PRICE])
# Make sure the price was paid.
assert (df[C.PRICE] == df[C.PAID]).all()

In [ ]:
envelopes = (
    df.pivot_table(
        index=[C.GRADE, C.HOMEROOM, C.STUDENT_NUMBER, C.STUDENT_NAME],
        columns=C.ITEM_NAME,
        values=C.PRICE,
        aggfunc="count",
    )
    .fillna("")
    .map(lambda x: x if x == "" else int(x))
)
envelopes = envelopes.rename(
    columns=lambda x: x.replace(
        "...  UHill Elementary PAC Grocery Card Fundraiser ... ", ""
    )
)
envelopes = envelopes[sorted(envelopes.columns)]
# Old: envelopes = envelopes.sort_values([C.HOMEROOM, C.STUDENT_NAME])
# Corinna prefers sorting by student name to make for easier pickup.
envelopes = envelopes.sort_values([C.STUDENT_NAME, C.STUDENT_NUMBER])
envelopes[C.ENVELOPE_NUMBER] = np.arange(len(envelopes.index))
envelopes

In [ ]:
envelopes.reset_index().to_excel(
    "/Users/thad/Dropbox/thad/U-Hill PAC/2025_Q4/envelopes.xlsx"
)

In [ ]:
envelopes.query(f"`{C.STUDENT_NAME}` == 'Hughes, Conrad'").reset_index().to_excel(
    "thad/uhill_pac/grocery_card_artifacts/conrad_only.xlsx"
)

# Assign cards to envelopes

In [ ]:
student_to_envelope = envelopes.reset_index()[
    [C.STUDENT_NUMBER, C.ENVELOPE_NUMBER]
].set_index(C.STUDENT_NUMBER)

In [ ]:
assigned = df.merge(student_to_envelope, left_on=C.STUDENT_NUMBER, right_index=True)
assigned = assigned.sort_values([C.ENVELOPE_NUMBER, C.ITEM_NAME])
assigned.head(20)

In [ ]:
assigned[assigned[C.STUDENT_NUMBER].isin([-1, -2])]

In [ ]:
!open ~/Dropbox/thad/gdrive/School/UHill\ Elementary/U-Hill\ PAC/2025.02\ -\ Fundraiser/Choices\ invoice.pdf

In [ ]:
choices_50 = """- Card 6275??????????????1 Successfully Activated ($50.00)
- Card 6275??????????????2 Successfully Activated ($50.00)""".splitlines()
choices_50 = [x.split(" ")[2] for x in choices_50]
choices_50
#

choices_100 = (
    """- Card 6275330202500879304 Successfully Activated ($100.00)""".splitlines()
)
choices_100 = [x.split(" ")[2] for x in choices_100]
choices_100

choices_200 = """- Card 6275330202500879312 Successfully Activated ($200.00)
- Card 6275330202500879320 Successfully Activated ($200.00)
- Card 6275330202500879338 Successfully Activated ($200.00)
- Card 6275330202500879346 Successfully Activated ($200.00)
- Card 6275330202500879353 Successfully Activated ($200.00)
- Card 6275330202500879361 Successfully Activated ($200.00)
- Card 6275330202500879379 Successfully Activated ($200.00)
- Card 6275330202500879387 Successfully Activated ($200.00)
- Card 6275330202500879395 Successfully Activated ($200.00)
- Card 6275330202500879403 Successfully Activated ($200.00)
- Card 6275330202500879411 Successfully Activated ($200.00)
- Card 6275330202500879429 Successfully Activated ($200.00)
- Card 6275330202500879437 Successfully Activated ($200.00)
- Card 6275330202500879445 Successfully Activated ($200.00)
- Card 6275330202500879452 Successfully Activated ($200.00)
- Card 6275330202500879460 Successfully Activated ($200.00)
- Card 6275330202500879478 Successfully Activated ($200.00)
- Card 6275330202500879486 Successfully Activated ($200.00)
- Card 6275330202500879494 Successfully Activated ($200.00)
- Card 6275330202500879551 Successfully Activated ($200.00)
- Card 6275330202500879569 Successfully Activated ($200.00)
- Card 6275330202500879577 Successfully Activated ($200.00)
- Card 6275330202500879585 Successfully Activated ($200.00)
- Card 6275330202500879593 Successfully Activated ($200.00)
- Card 6275330202500879601 Successfully Activated ($200.00)
- Card 6275330202500879619 Successfully Activated ($200.00)
- Card 6275330202500879627 Successfully Activated ($200.00)
- Card 6275330202500879635 Successfully Activated ($200.00)
- Card 6275330202500879643 Successfully Activated ($200.00)
- Card 6275330202500879650 Successfully Activated ($200.00)""".splitlines()
choices_200 = [x.split(" ")[2] for x in choices_200]

In [ ]:
save_on_50 = list(np.arange(631680792720, 631680792734 + 1))
save_on_100 = list(np.arange(633061889969, 633061890052 + 1))

# Merge together adjacent $100 cards to form virtual $200 cards
save_on_200_count = (
    2
    * totals.loc[
        "Save-On-Foods ...  UHill Elementary PAC Grocery Card Fundraiser ... $200",
        "Count",
    ]
)
save_on_200 = [
    f"{c1},{c2}"
    for c1, c2 in zip(
        save_on_100[0:save_on_200_count:2], save_on_100[1:save_on_200_count:2]
    )
]
save_on_100[:save_on_200_count] = []

In [ ]:
save_on_200, totals.loc[
    "Save-On-Foods ...  UHill Elementary PAC Grocery Card Fundraiser ... $200", "Count"
]

In [ ]:
totals.loc[
    "Choices Market ...  UHill Elementary PAC Grocery Card Fundraiser ... $200",
    "Count",
]

In [ ]:
assert (
    len(set(choices_50))
    == totals.loc[
        "Choices Market ...  UHill Elementary PAC Grocery Card Fundraiser ... $50",
        "Count",
    ]
), f"{len(set(choices_50))=} != {totals.loc['Choices Market ...  UHill Elementary PAC Grocery Card Fundraiser ... $50', 'Count']=}"

assert (
    len(set(choices_100))
    == totals.loc[
        "Choices Market ...  UHill Elementary PAC Grocery Card Fundraiser ... $100",
        "Count",
    ]
), f"{len(set(choices_100))=} != {totals.loc['Choices Market ...  UHill Elementary PAC Grocery Card Fundraiser ... $100', 'Count']=}"
assert (
    len(set(choices_200))
    == totals.loc[
        "Choices Market ...  UHill Elementary PAC Grocery Card Fundraiser ... $200",
        "Count",
    ]
), f"{len(set(choices_200))=} != {totals.loc['Choices Market ...  UHill Elementary PAC Grocery Card Fundraiser ... $200', 'Count']=}"

assert (
    len(set(save_on_50))
    == totals.loc[
        "Save-On-Foods ...  UHill Elementary PAC Grocery Card Fundraiser ... $50",
        "Count",
    ]
), f"{len(set(save_on_50))=} != {totals.loc['Save-On-Foods ...  UHill Elementary PAC Grocery Card Fundraiser ... $50', 'Count']=}"
assert (
    len(set(save_on_100))
    == totals.loc[
        "Save-On-Foods ...  UHill Elementary PAC Grocery Card Fundraiser ... $100",
        "Count",
    ]
), f"{len(set(save_on_100))=} != {totals.loc['Save-On-Foods ...  UHill Elementary PAC Grocery Card Fundraiser ... $100', 'Count']=}"
assert (
    len(set(save_on_200))
    == totals.loc[
        "Save-On-Foods ...  UHill Elementary PAC Grocery Card Fundraiser ... $200",
        "Count",
    ]
), f"{len(set(save_on_200))=} != {totals.loc['Save-On-Foods ...  UHill Elementary PAC Grocery Card Fundraiser ... $200', 'Count']=}"

print(f"Virtual $200 cards: {save_on_200=}")


ITEM_NAME_TO_NUMBERS = {
    "Choices Market ...  UHill Elementary PAC Grocery Card Fundraiser ... $200": choices_200,
    "Choices Market ...  UHill Elementary PAC Grocery Card Fundraiser ... $100": choices_100,
    "Choices Market ...  UHill Elementary PAC Grocery Card Fundraiser ... $50": choices_50,
    # "Choices ...  UHill Elementary PAC Grocery Card Fundraiser ... $50": choices_50,
    "Save-On-Foods ...  UHill Elementary PAC Grocery Card Fundraiser ... $100": save_on_100,
    "Save-On-Foods ...  UHill Elementary PAC Grocery Card Fundraiser ... $200": save_on_200,
    "Save-On-Foods ...  UHill Elementary PAC Grocery Card Fundraiser ... $50": save_on_50,
}


def assign_card(item_name) -> str:
    assert item_name in ITEM_NAME_TO_NUMBERS, f"No item name {item_name=}"
    assert ITEM_NAME_TO_NUMBERS[item_name], f"No cards left for {item_name=}"
    result = str(ITEM_NAME_TO_NUMBERS[item_name].pop(0))
    # print(f"{item_name=}, {result}")
    return result


assigned[C.CARD_NUMBER] = [assign_card(i) for i in assigned[C.ITEM_NAME]]

for name, numbers in ITEM_NAME_TO_NUMBERS.items():
    assert not numbers, f"Didn't use all cards for {name=}"

In [ ]:
envelope_contents = (
    assigned.groupby([C.ENVELOPE_NUMBER, C.STUDENT_NAME, C.ITEM_NAME])[C.CARD_NUMBER]
    .aggregate(lambda x: " ".join(x.values))
    .reset_index()
)
envelope_contents[C.ITEM_NAME] = envelope_contents[C.ITEM_NAME].str.replace(
    "...  UHill Elementary PAC Grocery Card Fundraiser ... ", ""
)
envelope_contents

In [ ]:
envelope_contents.to_excel(
    "thad/uhill_pac/grocery_card_artifacts/envelope_contents.xlsx"
)

In [ ]:
!open "thad/uhill_pac/grocery_card_artifacts/envelope_contents.xlsx"